## 1. Import packages

In [1]:
#!pip install tweepy

In [2]:
import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm

## 2. Twitter API authentication

In [3]:
consumer_api_key = os.environ["TWITTER_CONSUMER_API_KEY"]
consumer_api_secret = os.environ["TWITTER_CONSUMER_API_SECRET"]

In [4]:
auth = tw.OAuthHandler(consumer_api_key, consumer_api_secret)

In [5]:
api = tw.API(auth, wait_on_rate_limit=True)

## 3. Tweets query

### 3.1. Define the query

In [6]:
search_words = "#covid19 -filter:retweets"
date_since = "2020-03-01"
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(7500)

### 3.2. Retreive the tweets

In [7]:
tweets_copy = []
for tweet in tqdm(tweets):
    tweets_copy.append(tweet)

2780it [01:27, 31.60it/s]


In [8]:
print(f"new tweets retrieved: {len(tweets_copy)}")

new tweets retrieved: 2780


## 4. Populate the dataset

In [9]:
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'user_friends': tweet.user.friends_count,
                                               'user_favourites': tweet.user.favourites_count,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': tweet.text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source,
                                               'is_retweet': tweet.retweeted}, index=[0]))

100%|█████████████████████████████████████████████████████████████████████████████| 2780/2780 [00:10<00:00, 264.00it/s]


In [10]:
tweets_df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,Totally Toytastic,"South Molton, North Devon, UK",Totally Toytastic Ltd is an online toy store w...,2013-01-16 10:51:09,10104,11019,5975,False,2020-07-29 16:30:35,Fun Reusable Face Coverings for Adults and Kid...,"[facemask, face]",Instagram,False
0,DAILY SABAH,Istanbul,"Turkey’s top English newspaper, providing the ...",2013-12-05 19:08:07,485787,2,0,True,2020-07-29 16:30:35,New Orleans Pelicans to take on Utah Jazz as N...,[COVID19],Hootsuite Inc.,False
0,Peggy Smedley,"Carol Stream, IL","Tech-People-Connector: Podcast doer, influence...",2009-07-20 15:57:05,29731,3907,1268,False,2020-07-29 16:30:34,.@BW_Research &amp; .@e2org show some improvem...,[energy],Hootsuite Inc.,False
0,D.E. Sciulli 🎚,"The Laurel Highlands, PA, USA","Lib, Dem & Atheist⚡️Trump HATER👎 “I know it’s ...",2014-07-28 23:25:09,11973,12128,13524,False,2020-07-29 16:30:34,Maybe if a few high profile Republicans suffer...,[Covid19],Twitter for iPhone,False
0,McKnight's LTC News,"Northbrook, IL",An independent news resource for those in the ...,2009-06-29 19:48:13,10146,889,894,False,2020-07-29 16:30:34,"In her latest, @DrEl gathers advice from LTC p...",[COVID19],Hootsuite Inc.,False


## 5. Save the data

### 5.1. Read past data

In [11]:
tweets_old_df = pd.read_csv("covid19_tweets.csv")
print(f"past tweets: {tweets_old_df.shape}")

past tweets: (39676, 13)


### 5.2. Merge past and present data

In [12]:
tweets_all_df = pd.concat([tweets_old_df, tweets_df], axis=0)
print(f"new tweets: {tweets_df.shape[0]} past tweets: {tweets_old_df.shape[0]} all tweets: {tweets_all_df.shape[0]}")

new tweets: 2780 past tweets: 39676 all tweets: 42456


### 5.3. Drop duplicates

In [13]:
tweets_all_df.drop_duplicates(subset = ["user_name", "date", "text"], inplace=True)
print(f"all tweets: {tweets_all_df.shape}")

all tweets: (42456, 13)


### 5.4. Export the updated data

In [14]:
tweets_all_df.to_csv("covid19_tweets.csv", index=False)